In [47]:
import sqlite3
import pandas as pd
from typing import List

# Подключаемся к базе

In [48]:
conn = sqlite3.connect("data.db")

# Достаём поставщиков, у которых нет ИНН, но есть имя или ОГРН, по ковидным закупкам
(Нужна таблица covid_contracts из файла create_dataset.ipynb)

In [78]:
lot_suppliers = pd.read_sql_query(
    f"""
    SELECT lot_guid, supplier_guid, si2.inn as supplier_inn,
        si2.name as supplier_name, si2.ogrn as supplier_ogrn, si2.type as supplier_type
    FROM (
        SELECT lot_guid, MAX(supplier_guid) as supplier_guid
        FROM (
            SELECT c.lot_guid
            FROM covid_contracts c
            GROUP BY c.lot_guid
        ) tmp1
        INNER JOIN protocol_lot pl ON tmp1.lot_guid = pl.guid
        LEFT JOIN protocol_lot_applications pla on pl.guid = pla.protocol_lot_guid
        LEFT JOIN application a on pla.id = a.protocol_lot_applications_id
        LEFT JOIN supplier_info si on a.supplier_guid = si.guid
        WHERE (contract_signed = 1 or
         (a.winner_indication = 'Победитель'and contract_signed is null))
         and (supplier_guid is not null)
         and (si.inn is null and (si.name is not null or si.ogrn is not null))
        GROUP BY lot_guid
    ) sup
    LEFT JOIN supplier_info si2 ON si2.guid = sup.supplier_guid;
    """,
    conn
)

lot_suppliers

,lot_guid,supplier_guid,supplier_inn,supplier_name,supplier_ogrn,supplier_type
0,005056bd-6164-1eeb-9fb2-80f01f612591,2965f69d-c02b-42b2-91e5-478fd6adb776,None,Не предусмотрено.,None,Юридическое лицо
1,005056bd-6164-1eeb-a1ae-7ba36231c160,cf0fc835-edf4-4ea7-abe6-17cbbed0c26e,None,"ООО НПП ""Ареал""",None,Юридическое лицо
2,026fcfb8-4455-4b4d-883a-f617b00cdab3,35eccf7b-f3c3-4485-99de-040ed279cd37,None,ИП Гарифуллин А.Н.,None,Физическое лицо
3,0437d64f-ebdd-4655-9b47-4a8e72d87272,249271e5-72f2-444e-b285-1b9f48cb8af6,None,1,None,Юридическое лицо
4,04d6845a-2383-4c6a-b586-cb91a62b9643,a236aba8-b4dd-4d7d-adb7-8e5dac988792,None,ООО ИнтерЛабСервис,None,Юридическое лицо
...,...,...,...,...,...,...
200,facf8fed-230d-440b-aedf-f8f3b5314719,0ac08fb1-9a48-4bf5-a575-745bfca9b314,None,1,None,Юридическое лицо
201,fdccc2e6-448a-4c97-a08e-658d675133e7,20ba4f29-cf14-4194-b094-94d5ebc2f8b8,None,"Общество с ограниченной ответственностью ""ОллНэт""",None,Юридическое лицо
202,fe3ba8b8-8034-47a4-919d-deea81b45945,a4895213-6489-42cd-9138-2affff3ebd80,None,Заявка 1,None,Юридическое лицо
203,ff968e2f-ad32-9b54-1493-e52fff049a34,ff1b4bc7-3b7c-4c5b-820d-21eb9208996c,None,Участник 2,None,Физическое лицо


# Удалим анонимные заявки

In [79]:
anonym_names = {
    '1',
    'Заявка 1',
    'Участник 1',
    'Участник 4',
    'Участник 2',
    'Не предусмотрено.',
    'Заявка 2',
    'Участник 7',
    'Участник 5',
    'Участник №1',
    'Заявка 12',
    'Участник 3',
    'Заявка 5818',
    'Участник 6',
    'Участник 6'
    'Заявка 7880',
    'Заявка 7880',
    'Заявка 8097',
    'Заявка №12571',
    'не предоставляется'
}

def is_anonym_name(name: str) -> bool:
    return name in anonym_names

lot_suppliers['is_anonym'] = lot_suppliers['supplier_name'].map(is_anonym_name)
lot_suppliers = lot_suppliers[lot_suppliers['is_anonym'] == False]
lot_suppliers

,lot_guid,supplier_guid,supplier_inn,supplier_name,supplier_ogrn,supplier_type,is_anonym
1,005056bd-6164-1eeb-a1ae-7ba36231c160,cf0fc835-edf4-4ea7-abe6-17cbbed0c26e,None,"ООО НПП ""Ареал""",None,Юридическое лицо,False
2,026fcfb8-4455-4b4d-883a-f617b00cdab3,35eccf7b-f3c3-4485-99de-040ed279cd37,None,ИП Гарифуллин А.Н.,None,Физическое лицо,False
4,04d6845a-2383-4c6a-b586-cb91a62b9643,a236aba8-b4dd-4d7d-adb7-8e5dac988792,None,ООО ИнтерЛабСервис,None,Юридическое лицо,False
6,06510062-3df1-4f2a-85f7-2f548ea3ffeb,8afe21e7-d135-466f-b5c5-280c112ef055,None,"ООО ""АДР-Трафик""",None,Юридическое лицо,False
8,083e4f95-a0b7-7967-a72c-1c23babda3ce,deb4eb71-523f-4063-b16e-76e718140c23,None,Общество с ограниченной ответственностью «ДВ А...,1102537003953,Юридическое лицо,False
...,...,...,...,...,...,...,...
193,f118da07-418a-4f7b-bf0b-dae0b7739d82,d142a09a-ae65-42ae-b497-82c537bdc9e5,None,ООО ДНК Технология,None,Юридическое лицо,False
194,f1840e5c-15e6-4b48-8a32-9be7693685ad,dcba4e42-0611-472b-b8e3-edffce7bd6c8,None,"ГБУЗ ТО ""Областная клиническая больница №1"" г....",None,Юридическое лицо,False
198,f96c040a-5e28-44b5-9c7b-925f41a19a29,5ecd8d4d-f194-4bb2-a14b-2d194785d07c,None,ИП Тороян Дианна Шерамовна,None,Физическое лицо,False
201,fdccc2e6-448a-4c97-a08e-658d675133e7,20ba4f29-cf14-4194-b094-94d5ebc2f8b8,None,"Общество с ограниченной ответственностью ""ОллНэт""",None,Юридическое лицо,False


# Сгруппируем по имени

In [80]:
lot_suppliers.groupby('supplier_name').size()

supplier_name
ГАУЗ ТО «ОКБ №1»                                                                                                                                                   1
ГБУЗ ТО "ОКБ №1" г.Тюмень                                                                                                                                          1
ГБУЗ ТО "Областная клиническая больница №1" г.Тюмени                                                                                                               1
Голенкова Маргарита Николаевна                                                                                                                                     1
Государственное автономное учреждение Республики Саха (Якутия) «Республиканская клиническая больница №3»                                                           1
                                                                                                                                                                 

# Заполняем ИНН

In [90]:
missed = []

In [164]:
def list_to_query(str_list: List[str]) -> str:
    query_str = ""
    list_len = len(str_list)
    for _idx in range(list_len):
        query_str += f"'{str_list[_idx]}'"
        if _idx != list_len - 1:
            query_str += ','
    return query_str

def fill_supplier_inn(_names: List[str], _inn: str):
    query_names = list_to_query(_names)

    supplier = pd.read_sql_query(
        f"""
        SELECT * FROM supplier_info where name in ({query_names});
        """,
        conn
    )

    guid = ''
    present_inn = supplier[~supplier['inn'].isnull()]
    if present_inn.shape[0] > 0:
        guid = present_inn.iloc[0]['guid']
        m_inn = present_inn.iloc[0]['inn']
        if _inn != m_inn:
            print(f'ERROR {_inn} != {m_inn}')
            return
    else:
        guid = supplier.iloc[0]['guid']
        cur = conn.cursor()
        cur.execute(f"UPDATE supplier_info SET inn = '{_inn}' WHERE guid='{guid}'")
        conn.commit()
        cur.close()

    guids = supplier['guid'].tolist()
    query_guids = list_to_query(guids)

    cur = conn.cursor()
    cur.execute(f"UPDATE application SET supplier_guid = '{guid}' WHERE supplier_guid in ({query_guids})")
    conn.commit()
    cur.close()

### АО "Фармация"
https://www.rusprofile.ru/id/1601224

In [53]:
names = ['АО "Фармация"', 'Акционерное общество "Фармация"']
inn = "7202157342"
fill_supplier_inn(names, inn)

### АО «Медико-санитарная часть «Нефтяник»
https://www.rusprofile.ru/id/145266

In [77]:
names = [
    'АО «Медико-санитарная часть «Нефтяник»',
    'АО "Медико-санитарная часть "Нефтяник"',
    'АО "МСЧ "Нефтяник"',
    'АО «Медико-санитарная часть «Нефтяник» '
]
inn = "7202086042"
fill_supplier_inn(names, inn)

### Акционерное общество «Лента»
https://www.rusprofile.ru/id/1038748

In [58]:
names = ['Акционерное общество «Лента»']
inn = "2127008692"
fill_supplier_inn(names, inn)

### ГАУЗ ТО "ГОРОДСКАЯ ПОЛИКЛИНИКА №17" (г.Тюмень)
https://www.rusprofile.ru/id/2349989

In [59]:
names = ['ГАУЗ ТО "ГОРОДСКАЯ ПОЛИКЛИНИКА №17" (г.Тюмень)', 'ГАУЗ ТО "Городская поликлиника №17"']
inn = "7203116331"
fill_supplier_inn(names, inn)

### ГАУЗ ТО «ОКБ №1»
https://www.rusprofile.ru/id/9923137

In [81]:
names = [
    'ГАУЗ ТО «ОКБ №1»',
    'ГБУЗ ТО «ОКБ №1»               (г. Тюмень) ',
    'ГБУЗ ТО "ОБ №1" г.Тюмень',
    'ГБУЗ ТО "ОКБ №1" г.Тюмень',
    'ГАУЗ ТО "ОКБ №1"',
    'ГБУЗ ТО "Областная клиническая больница №1" г.Тюмени',
    'ГБУЗ ТО "Областная клиническая больница №1"',
    'Государственное бюджетное учреждение здравоохранения Тюменской области «Областная клиническая больница №1» ',
    'ГБУЗ ТО "ОКБ №1"'
]
inn = "7203000249"
fill_supplier_inn(names, inn)

### Голенкова Маргарита Николаевна
https://www.rusprofile.ru/ip/319325600000672

In [83]:
names = [
    'ИП Голенкова М.Н.',
    'Голенкова Маргарита Николаевна'
]
inn = "323408634627"
fill_supplier_inn(names, inn)

### Государственное автономное учреждение Республики Саха (Якутия) «Республиканская клиническая больница №3»
https://www.rusprofile.ru/id/4119281

In [84]:
names = [
    'Государственное автономное учреждение Республики Саха (Якутия) «Республиканская клиническая больница №3»',
    'ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ РЕСПУБЛИКИ САХА (ЯКУТИЯ) "РЕСПУБЛИКАНСКАЯ КЛИНИЧЕСКАЯ БОЛЬНИЦА №3"'
]
inn = "1435024023"
fill_supplier_inn(names, inn)

### ЗАО НПО «ГАРАНТ»
https://www.rusprofile.ru/id/3061194

In [85]:
names = [
    'ЗАО НПО «ГАРАНТ»',
    'ЗАО НПО "Гарант"',
    'ЗАО НПО «Гарант»'
]
inn = "7716017505"
fill_supplier_inn(names, inn)

### ИП Гарифуллин А.Н.
https://www.rusprofile.ru/ip/317784700241060

In [86]:
names = [
    'ИП Гарифуллин А.Н.',
]
inn = "450132791601"
fill_supplier_inn(names, inn)

### ИП Грицюк
Без имени отчества не определить точно
https://www.rusprofile.ru/ip/319774600160971
https://www.rusprofile.ru/ip/315774600032550

In [91]:
names = ['ИП Грицюк']
missed += names

### ИП Резников Дмитрий Александрович
Не определить точно
https://www.rusprofile.ru/ip/321774600007587
https://www.rusprofile.ru/ip/317723200001220

In [92]:
names = [
    'ИП Резников Дмитрий Александрович '
]
missed += names

### ИП Тороян Д.Ш.
https://www.rusprofile.ru/ip/304720732900059

In [87]:
names = [
    'ИП Тороян Дианна Шерамовна',
    'ИП Тороян Д.Ш.',
]
inn = "722800226122"
fill_supplier_inn(names, inn)

### Индивидуальный предприниматель Ситников Максим Сергеевич
Невозможно определить
https://www.rusprofile.ru/ip/321440000015832
https://www.rusprofile.ru/ip/321784700261540

In [94]:
names = [
    'Индивидуальный предприниматель Ситников Максим Сергеевич '
]
missed += names

### Индивидуальный предприниматель Солодкий Дмитрий Павлович
https://www.rusprofile.ru/ip/310526008400032

In [95]:
names = [
    'Индивидуальный предприниматель Солодкий Дмитрий Павлович'
]
inn = "526098514448"
fill_supplier_inn(names, inn)

### Индивидуальный предприниматель Тибирьков Сергей Евгеньевич
https://www.rusprofile.ru/ip/314345725900031

In [96]:
names = [
    'Тибирьков Сергей Евгеньевич',
    'Индивидуальный предприниматель Тибирьков Сергей Евгеньевич'
]
inn = "341700066482"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «Компания Экотех»
https://www.rusprofile.ru/id/4872060

In [98]:
names = [
    'Компания Экотех',
    'Общество с ограниченной ответственностью «Компания Экотех»'
]
inn = "7716681579"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Аква-сервис"
Невозможно определить
https://www.rusprofile.ru/id/667040
https://www.rusprofile.ru/id/6821822

In [99]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Аква-сервис"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Визирь"
Невозможно определить
https://www.rusprofile.ru/id/10642775
https://www.rusprofile.ru/id/3755163

In [100]:
names = [
    'ООО "Визирь"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Визирь"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Лазаревка"
https://www.rusprofile.ru/id/5529710

In [102]:
names = [
    'ООО "Лазаревка"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Лазаревка"'
]
inn = "7203267323"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФармЛайн"
https://www.rusprofile.ru/id/3329982

In [135]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФармЛайн"',
    'ООО "ФармЛайн"',
    'ООО "Фарм Лайн"'
]
inn = "7714511476"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЮС ГРАНД"
https://www.rusprofile.ru/id/11908433

In [139]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЮС ГРАНД"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЮС ГРАНД"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Юс Гранд"',
    'ООО «Юс Гранд»',
    'Общество с ограниченной ответственностью "ЮС ГРАНД"',
    'ООО "ЮС Гранд"'
]
inn = "7203490314"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ТОРГОВАЯ ГРУППА "МЯСНОЙ ДВОР"
https://www.rusprofile.ru/id/3768805

In [107]:
names = [
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ТОРГОВАЯ ГРУППА "МЯСНОЙ ДВОР"',
    'Общество с ограниченной ответственность «Торговая группа «Мясной двор»'
]
inn = "5504071730"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЭНЕРГОСТРОЙ"
https://www.rusprofile.ru/id/7144463

In [108]:
names = [
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЭНЕРГОСТРОЙ"',
]
inn = "7733866971"
fill_supplier_inn(names, inn)

### ООО "АДР-Трафик"
https://www.rusprofile.ru/id/1207700376950

In [110]:
names = [
    'ООО "АДР-Трафик"',
]
inn = "9709066022"
fill_supplier_inn(names, inn)

### ООО "Аксиома БИО"
https://www.rusprofile.ru/id/7870899

In [112]:
names = [
    'ООО "Аксиома БИО"',
    'Общество с ограниченной ответственностью «АксиомаБИО»'
]
inn = "7724330020"
fill_supplier_inn(names, inn)

### ООО "Альхон -Трейд"
https://www.rusprofile.ru/id/11614943

In [113]:
names = [
    'ООО "Альхон -Трейд"',
    'ООО «Альхон-Трейд»',
    'Общество с ограниченной ответственностью "Альхон-Трейд"'
]
inn = "7203462204"
fill_supplier_inn(names, inn)

### ООО "Альянс-Сервис"
https://www.rusprofile.ru/id/3785475

In [114]:
names = [
    'ООО "Альянс-Сервис"'
]
inn = "7710705784"
fill_supplier_inn(names, inn)

### ООО "ВЕРБА"
Невозможно определить
https://www.rusprofile.ru/company/ooo-verba

In [115]:
names = [
    'ООО "ВЕРБА"'
]
missed += names

### ООО "ВЭС"
Невозможно определить
https://www.rusprofile.ru/id/5913281
https://www.rusprofile.ru/id/3657772
https://www.rusprofile.ru/id/3926182

In [117]:
names = [
    'ООО "ВЭС"'
]
missed += names

### ООО "КомСпецТех"
Невозможно определить
https://www.rusprofile.ru/id/6076474
https://www.rusprofile.ru/id/3650602

In [118]:
names = [
    'ООО "КомСпецТех"'
]
missed += names

### ООО "Лизоформ-СПб"
https://www.rusprofile.ru/id/973317

In [119]:
names = [
    'ООО "Лизоформ-СПб"'
]
inn = "7814108542"
fill_supplier_inn(names, inn)

### ООО "МедФарм"
Невозможно определить
https://www.rusprofile.ru/id/7845896
https://www.rusprofile.ru/id/10905675

In [120]:
names = [
    'ООО "МедФарм"'
]
missed += names

### ООО "МедиАрт"
Невозможно определить
https://www.rusprofile.ru/id/2738713
https://www.rusprofile.ru/id/11854933

In [179]:
names = [
    'ООО "МедиАрт"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "МедиАрт"',
    'ООО"МедиАрт"'
]
missed += names

### ООО "Мединтелл"
https://www.rusprofile.ru/id/1571132

In [122]:
names = [
    'Общество с ограниченной ответственностью "Мединтелл"',
    'ООО "Мединтелл"',
    'ООО «Мединтелл»',
    'Общество с ограниченной ответственностью «Мединтелл»',
    'ООО "Мединтел"'
]
inn = "7202062972"
fill_supplier_inn(names, inn)

### ООО "Морозко"
https://www.rusprofile.ru/id/1103928

In [124]:
names = [
    'ООО "Морозко"'
]
inn = "7728024000"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ПРАЙДГРУПП ДВ"
https://www.rusprofile.ru/id/11849977

In [126]:
names = [
    'ООО "ППАЙДГРУПП ДВ"'
]
inn = "2723206455"
fill_supplier_inn(names, inn)

### ООО "ПрофЭнерго"
Невозможно определить
https://www.rusprofile.ru/id/9458634
https://www.rusprofile.ru/id/10501997

In [127]:
names = [
    'ООО "ПрофЭнерго"'
]
missed += names

### ООО "СВ-Инжиниринг М"
https://www.rusprofile.ru/id/11134082

In [128]:
names = [
    'Общество с ограниченной ответственностью «СВ-Инжиниринг М» ',
    'ООО "СВ-Инжиниринг М"'
]
inn = "7733322643"
fill_supplier_inn(names, inn)

### ООО "СибПромГаз"
Невозможно определить
https://www.rusprofile.ru/id/3794392
https://www.rusprofile.ru/id/1205400028239

In [129]:
names = [
    'ООО "СибПромГаз"'
]
missed += names

### ООО "ТПК "ПЛОДОРОДИЕ"
https://www.rusprofile.ru/id/7850830

In [131]:
names = [
    'ООО "ТПК "ПЛОДОРОДИЕ"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ТПК "ПЛОДОРОДИЕ"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ТОРГОВО-ПРОИЗВОДСТВЕННАЯ КОМПАНИЯ "ПЛОДОРОДИЕ"'
]
inn = "7203357513"
fill_supplier_inn(names, inn)

### ООО "Тюменское АТП"
https://www.rusprofile.ru/id/4261942

In [133]:
names = [
    'ООО "Тюменское АТП"',
    'ООО "Тюменское автотранспортное предприятие"'
]
inn = "7203141970"
fill_supplier_inn(names, inn)

### ООО "Универсал"
Невозможно определить
https://www.rusprofile.ru/id/7313341
https://www.rusprofile.ru/id/1217700123695
https://www.rusprofile.ru/id/10357015

In [134]:
names = [
    'ООО "Универсал"'
]
missed += names

### ООО "Фармплюс"
https://www.rusprofile.ru/id/11907388

In [136]:
names = [
    'ООО «Фармплюс»',
    'ООО "Фармплюс"'
]
inn = "6658529147"
fill_supplier_inn(names, inn)

### ООО "Энергосервиис"
Невозможно определить
https://www.rusprofile.ru/id/5444108
https://www.rusprofile.ru/id/5850085

In [137]:
names = [
    'ООО "Энергосервиис"'
]
missed += names

### ООО "Лаборатория Медилис"
https://www.rusprofile.ru/id/341280

In [141]:
names = [
    'ООО "Лаборатория МедиЛИС"',
    'ООО 2лаборатория МедиЛИС"'
]
inn = "7726625051"
fill_supplier_inn(names, inn)

### ООО «1П Технолоджиз»
https://www.rusprofile.ru/id/6908865

In [143]:
names = [
    'ООО "1П Технолоджиз"',
    'ООО «1П Технолоджиз»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "1П ТЕХНОЛОДЖИЗ"'
]
inn = "7714905135"
fill_supplier_inn(names, inn)

### ООО «Альянс-Сервис Салават»
https://www.rusprofile.ru/id/11964696

In [145]:
names = [
    'ООО «Альянс-Сервис Салават»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "АЛЬЯНС-СЕРВИС САЛАВАТ"'
]
inn = "0274953083"
fill_supplier_inn(names, inn)

### ООО «Диагностические системы-Урал»
https://www.rusprofile.ru/id/6584165

In [146]:
names = [
    'ООО "Диагностические системы-Урал"',
    'ООО «Диагностические системы-Урал»'
]
inn = "6685024025"
fill_supplier_inn(names, inn)

### ООО «Евроген Лаб»
https://www.rusprofile.ru/id/2904166

In [147]:
names = [
    'ООО "Евроген Лаб"',
    'ООО «Евроген Лаб»'
]
inn = "7728650342"
fill_supplier_inn(names, inn)

### ООО «МЕДИКОМ»
Невозможно определить
https://www.rusprofile.ru/id/4005650
https://www.rusprofile.ru/id/4334735

In [149]:
names = [
    'ООО «МЕДИКОМ» '
]
missed += names

### ООО «МедПартнер»
Невозможно определить
https://www.rusprofile.ru/id/11930686
https://www.rusprofile.ru/id/7764744

In [151]:
names = [
    'ООО «МедПартнер»'
]
missed += names

### ООО «НХРС»
https://www.rusprofile.ru/id/1877388

In [152]:
names = [
    'ООО «НХРС',
    'ООО «НХРС»',
    'ООО «НХРС» ',
    'ООО «НХРС»,',
    'ООО НХРС'
]
inn = "6125028059"
fill_supplier_inn(names, inn)

### ООО «Научно-методический центр клинической лабораторной диагностики Ситилаб»
https://www.rusprofile.ru/id/5417508

In [154]:
names = [
    'ООО «Научно-методический центр клинической лабораторной диагностики Ситилаб»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "НАУЧНО-МЕТОДИЧЕСКИЙ ЦЕНТР КЛИНИЧЕСКОЙ ЛАБОРАТОРНОЙ ДИАГНОСТИКИ СИТИЛАБ"'
]
inn = "7714822471"
fill_supplier_inn(names, inn)

### ООО «РТК-Инвест
https://www.rusprofile.ru/id/11471403

In [156]:
names = [
    'ООО «РТК-Инвест  ',
    'ООО «РТК-Инвест»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "РТК-ИНВЕСТ"'
]
inn = "7726434025"
fill_supplier_inn(names, inn)

### ООО " Русал Медицинский Центр "
https://www.rusprofile.ru/id/225224

In [157]:
names = [
    'ООО «РУСАЛ Медицинский Центр»'
]
inn = "2465094460"
fill_supplier_inn(names, inn)

### ООО «Радника»
https://www.rusprofile.ru/id/6709045

In [158]:
names = [
    'ООО «Радника»'
]
inn = "2543023195"
fill_supplier_inn(names, inn)

### ООО «СТАТУС-РЕГИОН»
Невозможно определить
https://www.rusprofile.ru/id/3115817
https://www.rusprofile.ru/id/11703691

In [159]:
names = [
    'ООО «СТАТУС-РЕГИОН»'
]
missed += names

### ООО Авиа-1
https://www.rusprofile.ru/id/11221940

In [161]:
names = [
    'ООО Авиа-1'
]
inn = "9705111387"
fill_supplier_inn(names, inn)

### ООО Апоцентр
Невозможно определить
https://www.rusprofile.ru/id/1784630
https://www.rusprofile.ru/id/301648

In [162]:
names = [
    'ООО Апоцентр'
]
missed += names

### ООО ДНК Технология
https://www.rusprofile.ru/id/2749627

In [163]:
names = [
    'ООО "ДНК-Технология"',
    'ООО ДНК Технология',
    'ООО ДНК-Технология'
]
inn = "7723537840"
fill_supplier_inn(names, inn)

### ООО ИнтерЛабСервис
https://www.rusprofile.ru/id/630939

In [165]:
names = [
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ИНТЕРЛАБСЕРВИС"',
    'ООО  "ИнтерЛабСервис"',
    'ООО "ИнтерЛабСервис"',
    'ООО ИнтерЛабСервис'
]
inn = "7720263905"
fill_supplier_inn(names, inn)

### ООО Компания ПанЭко
https://www.rusprofile.ru/id/3562008

In [167]:
names = [
    'ООО Компания ПанЭко',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "КОМПАНИЯ "ПАНЭКО"'
]
inn = "7723613788"
fill_supplier_inn(names, inn)

### ООО НПП "Ареал"
https://www.rusprofile.ru/id/10638707

In [169]:
names = [
    'ООО НПП "Ареал"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ НАУЧНО-ПРОИЗВОДСТВЕННОЕ ПРЕДПРИЯТИЕ "АРЕАЛ"',
    'ООО НПП "АРЕАЛ"'
]
inn = "0277915495"
fill_supplier_inn(names, inn)

### ООО Ормет
https://www.rusprofile.ru/id/4214586

In [171]:
names = [
    'ООО Ормет'
]
inn = "7704547804"
fill_supplier_inn(names, inn)

### ООО Растер
https://www.rusprofile.ru/id/3295495

In [172]:
names = [
    'ООО "Растер"',
    'ООО Растер'
]
inn = "6658037917"
fill_supplier_inn(names, inn)

### ООО Торговый Дом «Артекс»
https://www.rusprofile.ru/id/7899899

In [174]:
names = [
    'ООО Торговый Дом «Артекс» ',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ТОРГОВЫЙ ДОМ "АРТЕКС"'
]
inn = "3702121280"
fill_supplier_inn(names, inn)

### ООО Элтемикс
https://www.rusprofile.ru/id/7446829

In [175]:
names = [
    'Общество с ограниченной ответственностью «Элтемикс»',
    'ООО Элтемикс'
]
inn = "3666197034"
fill_supplier_inn(names, inn)

### ООО Юго-Запад
Невозможно определить
https://www.rusprofile.ru/id/6909642
https://www.rusprofile.ru/id/2169570

In [176]:
names = [
    'ООО Юго-Запад'
]
missed += names

### ООО фирма «Тритон-ЭлектроникС»  г. Екатеринбург
https://www.rusprofile.ru/id/2415060

In [177]:
names = [
    'ООО фирма «Тритон-ЭлектроникС»  г. Екатеринбург'
]
inn = "6659005570"
fill_supplier_inn(names, inn)

### ООО"Экострой"
https://www.rusprofile.ru/id/7116995

In [182]:
names = [
    'ООО "Экострой"',
    'ООО"Экострой"',
    'Общество с ограниченной ответственностью “Экострой”'
]
inn = "5032269606"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «МЕДИПАЛ-ОНКО»
https://www.rusprofile.ru/id/879084

In [184]:
names = [
    'Общество с ограниченной ответственностью         «МЕДИПАЛ-ОНКО»',
    'ООО "Медипал-Онко" ',
    'Общество с ограниченной ответственностью «Медипал-онко»',
    'Общество с ограниченной ответственностью «Медипал-ОНКО»'
]
inn = "7701213835"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью "ОллНэт"
https://www.rusprofile.ru/id/2514066

In [185]:
names = [
    'Общество с ограниченной ответственностью "ОллНэт"',
    'ООО "ОллНэт"'
]
inn = "5047111570"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «БионаФарм»
https://www.rusprofile.ru/id/235398

In [186]:
names = [
    'Общество с ограниченной ответственностью «БионаФарм»',
    'ООО "БионаФарм"'
]
inn = "7725628067"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «Виталон»
https://www.rusprofile.ru/id/10830392

In [188]:
names = [
    'Общество с ограниченной ответственностью «Виталон»',
    'Общество с ограниченной ответственностью "ВИТАЛОН"',
    'ООО «ВИТАЛОН»'
]
inn = "5017112368"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «ДВ АльфаРесурс»
https://www.rusprofile.ru/id/657468

In [189]:
names = [
    'Общество с ограниченной ответственностью «ДВ АльфаРесурс»'
]
inn = "2537083622"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «ОБЛАСТНАЯ СЛУЖБА ДЕЗИНФЕКЦИИ»
https://www.rusprofile.ru/id/10814954

In [190]:
names = [
    'Общество с ограниченной ответственностью «ОБЛАСТНАЯ СЛУЖБА ДЕЗИНФЕКЦИИ» '
]
inn = "7203405333"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «Универ Клаб»
https://www.rusprofile.ru/id/5919105

In [193]:
names = [
    'Общество с ограниченной ответственностью «Универ Клаб»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "УНИВЕР КЛАБ"'
]
inn = "7728789672"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью Производственно-торговая компания «Профессиональный Лидер»
https://www.rusprofile.ru/id/10315641

In [194]:
names = [
    'Общество с ограниченной ответственностью Производственно-торговая компания «Профессиональный Лидер»',
    'Общество с ограниченной ответственностью Производственно-торговая компания «Профессиональный Лидер» '
]
inn = "7203370313"
fill_supplier_inn(names, inn)

### Обществом с ограниченной ответственностью «Эколайф»
Невозможно определить
https://www.rusprofile.ru/id/11475993
https://www.rusprofile.ru/id/6914001
https://www.rusprofile.ru/id/7099261

In [195]:
names = [
    'Обществом с ограниченной ответственностью «Эколайф»'
]
missed += names

### Пителин Сергей Михайлович
https://www.rusprofile.ru/ip/317470400078813

In [196]:
names = [
    'Пителин Сергей Михайлович ',
    'Пителин Сергей Михайлович'
]
inn = "301505252144"
fill_supplier_inn(names, inn)

### Погорелко Владимир Иванович
На руспрофайле нет, взято из БД

In [197]:
names = [
    'Погорелко Владимир Иванович',
]
inn = "551400222527"
fill_supplier_inn(names, inn)

### ФБУ НАУКИ "ЦЕНТРАЛЬНЫЙ НАУЧНО-ИССЛЕДОВАТЕЛЬСКИЙ ИНСТИТУТ ЭПИДЕМИОЛОГИИ" ФЕДЕРАЛЬНОЙ СЛУЖБЫ ПО НАДЗОРУ В СФЕРЕ ЗАЩИТЫ ПРАВ ПОТРЕБИТЕЛЕЙ И БЛАГОПОЛУЧИЯ ЧЕЛОВЕКА
https://www.rusprofile.ru/id/2873325

In [199]:
names = [
    'ФБУ НАУКИ "ЦЕНТРАЛЬНЫЙ НАУЧНО-ИССЛЕДОВАТЕЛЬСКИЙ ИНСТИТУТ ЭПИДЕМИОЛОГИИ" ФЕДЕРАЛЬНОЙ СЛУЖБЫ ПО НАДЗОРУ В СФЕРЕ ЗАЩИТЫ ПРАВ ПОТРЕБИТЕЛЕЙ И БЛАГОПОЛУЧИЯ ЧЕЛОВЕКА ',
    'ФБУН ЦНИИ Эпидемиологии Роспотребнадзора'
]
inn = "7720024671"
fill_supplier_inn(names, inn)

# Проверим, что заполнили всех поставщиков

In [200]:
lot_suppliers = pd.read_sql_query(
    f"""
    SELECT lot_guid, supplier_guid, si2.inn as supplier_inn,
        si2.name as supplier_name, si2.ogrn as supplier_ogrn, si2.type as supplier_type
    FROM (
        SELECT lot_guid, MAX(supplier_guid) as supplier_guid
        FROM (
            SELECT c.lot_guid
            FROM covid_contracts c
            GROUP BY c.lot_guid
        ) tmp1
        INNER JOIN protocol_lot pl ON tmp1.lot_guid = pl.guid
        LEFT JOIN protocol_lot_applications pla on pl.guid = pla.protocol_lot_guid
        LEFT JOIN application a on pla.id = a.protocol_lot_applications_id
        LEFT JOIN supplier_info si on a.supplier_guid = si.guid
        WHERE (contract_signed = 1 or
         (a.winner_indication = 'Победитель'and contract_signed is null))
         and (supplier_guid is not null)
         and (si.inn is null and (si.name is not null or si.ogrn is not null))
        GROUP BY lot_guid
    ) sup
    LEFT JOIN supplier_info si2 ON si2.guid = sup.supplier_guid;
    """,
    conn
)
lot_suppliers['is_anonym'] = lot_suppliers['supplier_name'].map(is_anonym_name)
lot_suppliers = lot_suppliers[lot_suppliers['is_anonym'] == False]

def is_missed(name: str) -> bool:
    return name in missed

lot_suppliers['is_missed'] = lot_suppliers['supplier_name'].map(is_missed)
lot_suppliers = lot_suppliers[lot_suppliers['is_missed'] == False]

lot_suppliers.groupby('supplier_name').size()

Series([], dtype: int64)